In [4]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error as MSE

#### Model 1: Poisson GLM

##### Step 1: Variable Selection

In [2]:
train=pd.read_csv('data/model_train.csv', index_col=0)
train.head()

,Gender,DrivAge,VehYear,VehModel,VehGroup,Area,State,Expo,Cnt,Amt,Freq,Sev,LossCost,cls_0
119891,Female,26-35,2006.0,Vw - Volkswagen - Fox City 1.0mi/ 1.0mi Total ...,Vw Volkswagen Fox 1.0,Piaui,Piaui,0.54,0.0,0.0,0.0,NaN,0.0,Piaui
584908,Male,>55,2007.0,Renault - Megane Grand Tour Dynam. Hi-flex 1.6...,Renault Megane,Grande Campinas,Sao Paulo,1.00,0.0,0.0,0.0,NaN,0.0,Sao Paulo
1125710,Female,>55,2004.0,Renault - Clio Dynamique 1.6 16v 110cv 3p,Renault Clio Acima De 1.0,Met. Porto Alegre e Caxias do Sul,Rio Grande do Sul,1.00,0.0,0.0,0.0,NaN,0.0,Rio Grande do Sul
1121310,Male,36-45,2008.0,Vw - Volkswagen - Spacefox 1.6 Total Flex 8v 4p,Vw Volkswagen Spacefox,Para,Para,1.00,0.0,0.0,0.0,NaN,0.0,Para
1207354,Male,>55,2006.0,Gm - Chevrolet - Zafira Elite 2.0 Mpfi Flexpow...,Gm Chevrolet Zafira,Grande Campinas,Sao Paulo,0.40,0.0,0.0,0.0,NaN,0.0,Sao Paulo


In [31]:
# Use 5 fold validation for variable selection 
expr_list=[]
Po_scores=[]
cv=[]

# Prepare GLM model formula
x_col=['Gender', 'DrivAge', 'VehYear']
## 3 terms crossed
expr_list.append((' * '.join(x_col)))
expr_list.append((' + '.join(x_col)))
## 2 terms crossed
for i, c in enumerate(x_col):
    expr_list.append(c + ' + ' + (' * '.join([x_col[_] for _ in range(len(x_col)) if _!=i])))
    expr_list.append((' * '.join([x_col[_] for _ in range(len(x_col)) if _!=i])))
## no terms crossed
for i, c in enumerate(x_col):
    expr_list.append(c)
    expr_list.append((' + '.join([x_col[_] for _ in range(len(x_col)) if _!=i])))

# Set up k-fold cross-validation
k_folds = 5
kf = KFold(n_splits=k_folds, shuffle=True, random_state=35)

for i, [train_index, test_index] in enumerate(kf.split(train)):
    print('FOLD', str(i))
    train_data = train.iloc[train_index]
    test_data = train.iloc[test_index]
    for id_exp, expr in enumerate(expr_list):
        cv.append(i)
        print(id_exp, expr) 
        FreqPoisson=sm.GLM.from_formula('Cnt ~ ' + expr, data=train_data, offset=np.log(train_data['Expo']), 
                            family=sm.families.Poisson(link=sm.families.links.Log())).fit()
        y_pred = FreqPoisson.predict(test_data, offset=np.log(test_data['Expo']))
        # save scores from each model
        score=np.array([FreqPoisson.deviance, FreqPoisson.bic_llf, FreqPoisson.aic], dtype=np.float32)
        Po_scores.append(list(score))
   

FOLD 0
0 Gender * DrivAge * VehYear
1 Gender + DrivAge + VehYear
2 Gender + DrivAge * VehYear
3 DrivAge * VehYear
4 DrivAge + Gender * VehYear
5 Gender * VehYear
6 VehYear + Gender * DrivAge
7 Gender * DrivAge
8 Gender
9 DrivAge + VehYear
10 DrivAge
11 Gender + VehYear
12 VehYear
13 Gender + DrivAge
FOLD 1
0 Gender * DrivAge * VehYear
1 Gender + DrivAge + VehYear
2 Gender + DrivAge * VehYear
3 DrivAge * VehYear
4 DrivAge + Gender * VehYear
5 Gender * VehYear
6 VehYear + Gender * DrivAge
7 Gender * DrivAge
8 Gender
9 DrivAge + VehYear
10 DrivAge
11 Gender + VehYear
12 VehYear
13 Gender + DrivAge
FOLD 2
0 Gender * DrivAge * VehYear
1 Gender + DrivAge + VehYear
2 Gender + DrivAge * VehYear
3 DrivAge * VehYear
4 DrivAge + Gender * VehYear
5 Gender * VehYear
6 VehYear + Gender * DrivAge
7 Gender * DrivAge
8 Gender
9 DrivAge + VehYear
10 DrivAge
11 Gender + VehYear
12 VehYear
13 Gender + DrivAge
FOLD 3
0 Gender * DrivAge * VehYear
1 Gender + DrivAge + VehYear
2 Gender + DrivAge * VehYear
3 D

In [47]:
# calculate the average of scores for each model
dfScore=pd.concat([
    pd.DataFrame({'model':np.tile(expr_list, k_folds)}),    
    pd.DataFrame(Po_scores, columns=['PO_deviance','PO_bic','PO_aic'])
    ], axis=1)
dfScore=dfScore.groupby('model').mean().sort_index()
#display(dfScore)
print('model index with the lowest validation score')
id_min=dfScore.iloc[:,0:].idxmin().drop_duplicates()
dfScore_min=dfScore.loc[id_min]
for c in dfScore_min.columns:
    dfScore_min[c+'_diff']=dfScore_min[c].diff()
display(dfScore_min)


model index with the lowest validation score


,PO_deviance,PO_bic,PO_aic,PO_deviance_diff,PO_bic_diff,PO_aic_diff
model,,,,,,
Gender * DrivAge * VehYear,358701.0000,595197.7500,594966.8750,NaN,NaN,NaN
VehYear + Gender * DrivAge,358721.1875,595096.0625,594969.0625,20.1875,-101.6875,2.1875


#### Selected 'VehYear + Gender * DrivAge' in the GLM model, which has the lowest BIC, and only +2 on AIC than the full model

##### Step 2: Fit Model 1 with the selected formula

In [5]:
expr_='Cnt ~ VehYear + Gender * DrivAge'
# use GLM residual as targets for Nnet
mod_p = smf.glm(formula=expr_, data=train, offset=np.log(train['Expo']), 
                 family=sm.families.Poisson(link=sm.families.links.Log()))
res_p=mod_p.fit()   
train['res']=res_p.resid_deviance

In [52]:
res_p.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                    Cnt   No. Observations:               951708
Model:                            GLM   Df Residuals:                   951697
Model Family:                 Poisson   Df Model:                           10
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -3.7184e+05
Date:                Tue, 22 Aug 2023   Deviance:                   4.4841e+05
Time:                        00:42:27   Pearson chi2:                 1.18e+06
No. Iterations:                     6   Pseudo R-squ. (CS):            0.01781
Covariance Type:            nonrobust                                         
===================================================================================================
                                      coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept                        -115.7966      1.230    -94.154      0.000    -118.207    -113.386
Gender[T.Male]                      0.3702      0.017     21.170      0.000       0.336       0.404
DrivAge[T.26-35]                   -0.2612      0.015    -17.694      0.000      -0.290      -0.232
DrivAge[T.36-45]                   -0.0801      0.014     -5.618      0.000      -0.108      -0.052
DrivAge[T.46-55]                   -0.2099      0.015    -14.173      0.000      -0.239      -0.181
DrivAge[T.>55]                     -0.3296      0.015    -21.474      0.000      -0.360      -0.300
Gender[T.Male]:DrivAge[T.26-35]    -0.2065      0.020    -10.532      0.000      -0.245      -0.168
Gender[T.Male]:DrivAge[T.36-45]    -0.2686      0.019    -14.331      0.000      -0.305      -0.232
Gender[T.Male]:DrivAge[T.46-55]    -0.4507      0.020    -22.973      0.000      -0.489      -0.412
Gender[T.Male]:DrivAge[T.>55]      -0.4265      0.020    -21.401      0.000      -0.466      -0.387
VehYear                             0.0564      0.001     92.148      0.000       0.055       0.058
===================================================================================================
"""

In [53]:
train.to_csv('data/GLM_1_output.csv')